# Statistik-Projekt HS25 – Notebook 01: Datenaufbereitung
## Ziel
Dieses Notebook erstellt die **saubere Datenbasis** für alle weiteren Analysen im Semesterprojekt.
Wir transformieren die Rohdaten der SBB (IST-Daten), bereinigen Fehler und erstellen spezifische Variablen für unsere drei Kern-Hypothesen:

1.  **Zeit-Hypothese:** Rush-Hour vs. Nebenverkehrszeit.
2.  **Typ-Hypothese:** Fernverkehr vs. S-Bahn/Regionalverkehr.
3.  **Wochenend-Hypothese:** Werktag vs. Wochenende.

## Input
- Datei: `../data/processed/istdata_real_2025-09.parquet` (Rohdaten)

## Output
- Datei: `../data/processed/istdata_clean_extended.parquet` (Bereinigt & Erweitert)

In [2]:
import polars as pl
import numpy as np
from pathlib import Path
import os

# Konfiguration der Pfade
INPUT_PATH = "../../data/processed/istdata_real_2025-09.parquet"
OUTPUT_PATH = "../../data/processed/istdata_clean_extended.parquet"

# Prüfen, ob Input existiert
if not Path(INPUT_PATH).exists():
    raise FileNotFoundError(f"Die Datei {INPUT_PATH} wurde nicht gefunden. Bitte Pfad prüfen!")

print("Setup erfolgreich. Bibliotheken geladen.")

Setup erfolgreich. Bibliotheken geladen.


In [3]:
# Wir nutzen scan_parquet für Lazy Loading (spart RAM bei großen Dateien)
lf = pl.scan_parquet(INPUT_PATH)

# Schema ausgeben, um Datentypen zu prüfen
print("--- Schema der Rohdaten ---")
print(lf.collect_schema())

# Kurzer Blick in die Daten (erste 5 Zeilen)
print("\n--- Vorschau (Head) ---")
display(lf.head(5).collect().to_pandas())

--- Schema der Rohdaten ---
Schema({'BETRIEBSTAG': String, 'BETREIBER_NAME': String, 'PRODUKT_ID': Categorical, 'LINIEN_TEXT': String, 'HALTESTELLEN_NAME': String, 'ANKUNFTSZEIT': Datetime(time_unit='ns', time_zone='UTC'), 'ABFAHRTSZEIT': Datetime(time_unit='ns', time_zone='UTC'), 'delay_arrival_s': Float64, 'delay_departure_s': Float64, 'on_time': Boolean, 'FAELLT_AUS_TF': Boolean, 'BPUIC': String})

--- Vorschau (Head) ---


,BETRIEBSTAG,BETREIBER_NAME,PRODUKT_ID,LINIEN_TEXT,HALTESTELLEN_NAME,ANKUNFTSZEIT,ABFAHRTSZEIT,delay_arrival_s,delay_departure_s,on_time,FAELLT_AUS_TF,BPUIC
0,01.09.2025,DB ZugBus Regionalverkehr Alb-Bodensee,Zug,IR3,Schaffhausen,2025-01-09 05:54:00+00:00,2025-01-09 05:56:00+00:00,114.0,78.0,False,False,8503424
1,01.09.2025,DB ZugBus Regionalverkehr Alb-Bodensee,Zug,RE3,Schaffhausen,2025-01-09 06:14:00+00:00,2025-01-09 06:16:00+00:00,42.0,1.0,True,False,8503424
2,01.09.2025,DB ZugBus Regionalverkehr Alb-Bodensee,Zug,IR3,Schaffhausen,2025-01-09 06:58:00+00:00,2025-01-09 07:00:00+00:00,31.0,17.0,True,False,8503424
3,01.09.2025,DB ZugBus Regionalverkehr Alb-Bodensee,Zug,RE3,Schaffhausen,2025-01-09 06:58:00+00:00,2025-01-09 07:00:00+00:00,18.0,-2.0,True,False,8503424
4,01.09.2025,DB ZugBus Regionalverkehr Alb-Bodensee,Zug,IR3,Schaffhausen,2025-01-09 07:58:00+00:00,2025-01-09 08:00:00+00:00,83.0,20.0,False,False,8503424


In [4]:
# Wir filtern und bereinigen die Daten in einem Schritt
lf_clean = (
    lf
    # 1. Filter: Wir analysieren nur ZÜGE (keine Busse, Schiffe, Trams)
    # Grund: Verspätungsdynamiken sind bei Bussen (Stau) komplett anders.
    .filter(pl.col("PRODUKT_ID") == "Zug")
    
    # 2. Verspätung berechnen (in Minuten)
    # Die Rohdaten haben Sekunden (delay_arrival_s). Wir wollen Minuten.
    .with_columns([
        (pl.col("delay_arrival_s") / 60).alias("arr_delay_min")
    ])
    
    # 3. Plausibilitäts-Filter
    # Wir entfernen extreme Ausreißer, die wahrscheinlich Messfehler sind.
    # Z.B. mehr als 2 Stunden zu früh (-120) oder mehr als 24h Verspätung.
    .filter(
        (pl.col("arr_delay_min") > -60) & 
        (pl.col("arr_delay_min") < 1440) # 24 Stunden
    )
    
    # 4. Missing Values (Optional: Zeilen ohne Ankunftszeit entfernen)
    .filter(pl.col("ANKUNFTSZEIT").is_not_null())
)

print("Cleaning-Logik definiert.")

Cleaning-Logik definiert.


In [5]:
lf_features = (
    lf_clean
    
    # ---------------------------------------------------------
    # HYPOTHESE 1: Tageszeit (Time Band)
    # Wir extrahieren die Stunde und ordnen sie Kategorien zu.
    # ---------------------------------------------------------
    .with_columns(
        pl.col("ANKUNFTSZEIT").dt.hour().alias("hour_of_day")
    )
    .with_columns(
        pl.when(pl.col("hour_of_day").is_in([0, 1, 2, 3, 4, 5])).then(pl.lit("Nacht"))
        .when(pl.col("hour_of_day").is_in([6, 7, 8, 9])).then(pl.lit("Morgenpeak"))
        .when(pl.col("hour_of_day").is_in([10, 11, 12, 13, 14, 15])).then(pl.lit("Tagesverkehr"))
        .when(pl.col("hour_of_day").is_in([16, 17, 18, 19])).then(pl.lit("Abendpeak"))
        .otherwise(pl.lit("Spätabend")) # 20-23 Uhr
        .alias("time_band")
    )
    
    # ---------------------------------------------------------
    # HYPOTHESE 2: Linientyp (Fernverkehr vs. Regional vs. S-Bahn)
    # Wir leiten den Typ aus dem Text der Linie ab (z.B. "IC 5" -> Fernverkehr)
    # ---------------------------------------------------------
    .with_columns(
        pl.when(pl.col("LINIEN_TEXT").str.contains("^(IC|IR|EC|EN|RJ|TGV|ICE)")).then(pl.lit("Fernverkehr"))
        .when(pl.col("LINIEN_TEXT").str.starts_with("S")).then(pl.lit("S-Bahn"))
        .when(pl.col("LINIEN_TEXT").str.starts_with("RE")).then(pl.lit("Regional"))
        .when(pl.col("LINIEN_TEXT").str.starts_with("R")).then(pl.lit("Regional"))
        .otherwise(pl.lit("Andere"))
        .alias("linientyp")
    )
    
    # ---------------------------------------------------------
    # HYPOTHESE 3: Wochentag (Wochenende vs. Werktag)
    # Wir prüfen das Datum. (Polars: Monday=1, Sunday=7)
    # WICHTIG: Wir müssen sicherstellen, dass 'BETRIEBSTAG' als Datum erkannt wird.
    # Falls es ein String ist "01.09.2025", müssen wir parsen.
    # ---------------------------------------------------------
    .with_columns(
        # Versuch, das Datum zu parsen (Format anpassen falls nötig DD.MM.YYYY)
        pl.col("BETRIEBSTAG").str.to_date("%d.%m.%Y").alias("date_parsed")
    )
    .with_columns(
        pl.col("date_parsed").dt.weekday().alias("weekday_num")
    )
    .with_columns(
        # Samstag (6) und Sonntag (7) sind Wochenende
        (pl.col("weekday_num") >= 6).alias("is_weekend")
    )
)

print("Feature Engineering Logik definiert.")

Feature Engineering Logik definiert.


In [6]:
# Wir führen die Berechnung jetzt aus (collect)
print("Führe Transformationen durch (das kann einen Moment dauern)...")
df_final = lf_features.collect()

# --- SANITY CHECKS ---
print("\n1. Verteilung Linientyp:")
print(df_final["linientyp"].value_counts())

print("\n2. Verteilung Time Band:")
print(df_final["time_band"].value_counts())

print("\n3. Verteilung Wochenende:")
print(df_final["is_weekend"].value_counts())

print("\n4. Fehlende Werte (Nulls) nach Aufbereitung:")
print(df_final.null_count())

# Speichern nur der wichtigen Spalten (um Speicherplatz zu sparen)
cols_export = [
    "BETRIEBSTAG", "date_parsed", "weekday_num", "is_weekend", # Zeit/Tag
    "ANKUNFTSZEIT", "hour_of_day", "time_band",                # Uhrzeit
    "LINIEN_TEXT", "linientyp",                                # Typ
    "HALTESTELLEN_NAME", "BPUIC",                              # Ort
    "arr_delay_min", "on_time", "FAELLT_AUS_TF"                # Target
]

# Filtern auf Spalten, die auch wirklich existieren (falls Tippfehler oben waren)
existing_cols = [c for c in cols_export if c in df_final.columns]

print(f"\nSpeichere {len(df_final)} Zeilen in {OUTPUT_PATH}...")
df_final.select(existing_cols).write_parquet(OUTPUT_PATH)
print("✅ ERFOLGREICH. Notebook 01 abgeschlossen.")

Führe Transformationen durch (das kann einen Moment dauern)...

1. Verteilung Linientyp:
shape: (4, 2)
┌─────────────┬─────────┐
│ linientyp   ┆ count   │
│ ---         ┆ ---     │
│ str         ┆ u32     │
╞═════════════╪═════════╡
│ Andere      ┆ 82284   │
│ S-Bahn      ┆ 2720310 │
│ Regional    ┆ 1104395 │
│ Fernverkehr ┆ 294607  │
└─────────────┴─────────┘

2. Verteilung Time Band:
shape: (5, 2)
┌──────────────┬─────────┐
│ time_band    ┆ count   │
│ ---          ┆ ---     │
│ str          ┆ u32     │
╞══════════════╪═════════╡
│ Abendpeak    ┆ 948331  │
│ Nacht        ┆ 270432  │
│ Spätabend    ┆ 710819  │
│ Morgenpeak   ┆ 922309  │
│ Tagesverkehr ┆ 1349705 │
└──────────────┴─────────┘

3. Verteilung Wochenende:
shape: (2, 2)
┌────────────┬─────────┐
│ is_weekend ┆ count   │
│ ---        ┆ ---     │
│ bool       ┆ u32     │
╞════════════╪═════════╡
│ false      ┆ 3147597 │
│ true       ┆ 1053999 │
└────────────┴─────────┘

4. Fehlende Werte (Nulls) nach Aufbereitung:
shape: (1, 19